In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_1_NonPath_Path_GN_2286.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     14.767060  10.829177  ...        0.032505        0.032749
1     11.728905   8.138424  ...        0.031626        0.032946
2      8.319776   2.773259  ...        0.031596        0.031098
3     10.255686   3.778411  ...        0.031188        0.031233
4      6.745884   2.698354  ...        0.031939        0.030060
...         ...        ...  ...             ...             ...
2281  12.828743   5.186088  ...        0.031884        0.033215
2282   6.040447   3.294790  ...        0.029505        0.027986
2283   3.997220   1.598888  ...        0.031122        0.031575
2284   3.739325   1.602568  ...        0.031110        0.031553
2285   9.393071   2.415361  ...        0.033175        0.032958

[2286 rows x 150 columns]
0       1
1       1
2       1
3       1
4       1
       ..
2281    2
2282    2
2283    2
2284    2
2285    2
Name: Class, Length: 2286, dtype: category
Categories (2, int64): [1, 2]


In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 2 - 1) = 1 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 283,586
Trainable params: 283,586
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
58/58 [==============================] - 36s 63ms/step - loss: 0.6842 - accuracy: 0.6010 - val_loss: 0.5978 - val_accuracy: 0.6769
Epoch 2/1000
58/58 [==============================] - 1s 13ms/step - loss: 0.6463 - accuracy: 0.6093 - val_loss: 0.6011 - val_accuracy: 0.6812
Epoch 3/1000
58/58 [==============================] - 1s 14ms/step - loss: 0.6242 - accuracy: 0.6259 - val_loss: 0.6073 - val_accuracy: 0.6921
Epoch 4/1000
58/58 [==============================] - 1s 13ms/step - loss: 0.6418 - accuracy: 0.6145 - val_loss: 0.6006 - val_accuracy: 0.6834
Epoch 5/1000
58/58 [==============================] - 1s 13ms/step - loss: 0.6406 - accuracy: 0.6166 - val_loss: 0.6034 - val_accuracy: 0.6878
Epoch 6/1000
58/58 [==============================] - 1s 13ms/step - loss: 0.6350 - accuracy: 0.6005 - val_loss: 0.6078 - val_accuracy: 0.6943
Epoch 7/1000
58/58 [==============================] - 1s 13ms/step - loss: 0.6501 - accuracy: 0.5934 - val_loss: 0.6040 - val_accuracy: 0.687

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.6460612691466083
test-acc = 0.6724890829694323
train-CFR =               precision    recall  f1-score   support

           0       0.66      0.66      0.66       945
           1       0.64      0.63      0.63       883

    accuracy                           0.65      1828
   macro avg       0.65      0.65      0.65      1828
weighted avg       0.65      0.65      0.65      1828

test-CFR =               precision    recall  f1-score   support

           0       0.70      0.66      0.68       243
           1       0.64      0.68      0.66       215

    accuracy                           0.67       458
   macro avg       0.67      0.67      0.67       458
weighted avg       0.67      0.67      0.67       458

